In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.figure12 import Figure12
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy

In [2]:
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0 #-0.01
goalReached = 1000

env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
# env = Figure12(5, 5, (4, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [3]:
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
adam_and_eve.append(init_team)

for adam in adam_and_eve:
    print(adam.learners)
    print('------')
    adam.init_search(env, [2, 3])
    print(adam.learners)
    for learner in adam.learners:
        print(learner.program.rule.action_set)
        print(learner.program.rule.region)

[]
------
new step-------
3
(0, 0)
new step-------
3
(0, 0)
new step-------
2
(0, 1)
new step-------
2
(0, 2)
new step-------
2
(0, 3)
new step-------
2
(0, 4)
new step-------
2
(0, 4)
[0, 0, 0, 4]
[2, 3]
[0, 0, 0, 4]


In [4]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

# init teams
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
init_team.init_search(env, [2, 3])
adam_and_eve.append(init_team)
win = False
for adam in adam_and_eve:
#     win = adam.search(env)
    data = [adam, win]
    searchManager.evaluate_team(data)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

for gen in range(numGens):
    if win:
        print(f'got to --> gen {gen+1}')
        break
    toEvaluateAfterGen = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
    # get it started
#     if gen == 0:
#         for adam in adam_and_eve:
#             win = adam.search(env)
#             data = [adam, win]
#             searchManager.evaluate_team(data)
#     else:
    for parent in searchManager.teamPool:
        # only run if the team has not yet won
        if parent not in searchManager.winners:
            print(f'Parent: {parent.id}')
            env.current_state = parent.start_state

            # make offspring from parent, then let that offspring make a new rule
            child = copy.deepcopy(parent)
            child.id = uuid.uuid4()
            win = child.search(env)
            print(f'Child: {child.id}')
            print('Start Child Regions --------')
            for learner in child.learners:
                print(learner.program.rule.region)
            print('End Child Regions --------')
            print(f'Child Fitness: {child.fitness}') 
            toEvaluateAfterGen.append([child, win])
        else:
            print(f'Skipping Team: {parent.id}')
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

new step-------
3
(0, 0)
new step-------
3
(0, 0)
new step-------
2
(0, 1)
new step-------
2
(0, 2)
new step-------
2
(0, 3)
new step-------
2
(0, 4)
new step-------
2
(0, 4)
[0, 0, 0, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: 79926961-2bcf-4d51-9d8a-cb1ba30cafcb
Start: (0, 4)
New real step: 
0
(0, 4)
Backtrackregioncehck: [0, 0, 0, 4]
New real step: 
0
(0, 3)
Backtrackregioncehck: [0, 0, 0, 3]
New real step: 
0
(1, 2)
New real step: 
0
(2, 2)
New real step: 
0
(3, 2)
New real step: 
0
(4, 2)
New real step: 
0
(4, 2)
Child: 5aa95edc-2bc9-4627-86f5-616ca1064e64
Start Child Regions --------
[0, 0, 0, 1]
[1, 2, 0, 3]
End Child Regions --------
Child Fitness: 0.77
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 1 Regions
Team: 1 --> 0.39
[0, 0, 0, 4]
Team: 2 --> 0.77
[0, 0, 0, 1]
[1, 2, 0, 3]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 2 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: 79926961-2bcf-4d51-9

2
(4, 3)
New real step: 
2
(4, 4)
New real step: 
2
(4, 4)
New real step: 
3
(4, 3)
New real step: 
3
(4, 2)
New real step: 
3
(4, 1)
New real step: 
3
(4, 0)
New real step: 
3
(4, 0)
Child: a7353129-114f-466a-ad08-a6a5feea82fa
Start Child Regions --------
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
End Child Regions --------
Child Fitness: 4.3
Parent: 29ed60aa-d5e1-4ea2-8e67-379d0eb8c4c9
Start: (0, 0)
New real step: 
1
(0, 0)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
New real step: 
1
(0, 0)
Backtrackregioncehck: [0, 0, 0, 4]
New real step: 
1
(0, 1)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
New real step: 
1
(0, 0)
Child: c0d15738-6dd3-4716-

[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
End Child Regions --------
Child Fitness: 6.979999999999999
Parent: 40f53a2b-6018-49bc-b7d5-79c51dd49de6
Start: (4, 0)
New real step: 
0
(4, 0)
Backtrackregioncehck: [0, 4, 1, 4]
New real step: 
1
(3, 2)
New real step: 
1
(2, 2)
New real step: 
1
(1, 2)
New real step: 
1
(0, 2)
New real step: 
1
(0, 2)
Child: 1027bf44-0fe2-43cc-9507-05513bce8797
Start Child Regions --------
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
End Child Regions --------
Child Fitness: 7.369999999999998
Parent: 39d38967-080d-4bfe-95a4-122ee2c3a960
Start: (4, 2)
New real step: 
2
(4, 3)
New real step: 
2
(4, 4)
New real step: 
2
(4, 4)
New real step: 
3
(4, 3)
N

Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
New real step: 
1
(0, 0)
Backtrackregioncehck: [0, 0, 0, 4]
New real step: 
1
(0, 1)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
New real step: 
1
(0, 0)
Child: 81cf9355-4abd-435f-a65c-4a669b59ce9c
Start Child Regions --------
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 1, 4]
[1, 0, 0, 0]
End Child Regions --------
Child Fitness: 7.869999999999998
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 19 Regions
Team: 1 --> 8.089999999999998
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 

[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
Team: 2 --> 9.849999999999998
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
Team: 3 --> 9.659999999999998
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 0, 3]
Team: 4 --> 9.659999999999998
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
Team: 5 --> 9.659999999999

[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
Team: 2 --> 11.619999999999997
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 3]
Team: 3 --> 11.619999999999997
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 3]
Team: 4 --> 11.619999999999997
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 

New real step: 
0
(4, 2)
Backtrackregioncehck: [0, 4, 0, 2]
New real step: 
0
(4, 1)
Backtrackregioncehck: [0, 4, 0, 1]
New real step: 
0
(4, 0)
New real step: 
0
(4, 0)
New real step: 
0
(4, 0)
New real step: 
0
(4, 0)
New real step: 
0
(4, 0)
New real step: 
0
(4, 0)
New real step: 
0
(4, 0)
Child: 0b5ad65d-73ce-4076-a56f-081776c689ad
Start Child Regions --------
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 12.509999999999998
Parent: b8c9f627-9d53-4278-a3e7-0a6fbd538d67
Start: (4, 0)
New real step: 
2
(4, 1)
New real step: 
2
(4, 2)
New real step: 
2
(4, 3)
New real step: 
2
(4, 4)
New real step: 
2
(4, 4)
New real step: 
3
(4, 3

[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 3]
End Child Regions --------
Child Fitness: 14.559999999999993
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 34 Regions
Team: 1 --> 14.949999999999998
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
Team: 2 --> 14.559999999999997
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 

[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 1, 4]
Team: 3 --> 15.929999999999998
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
Team: 4 --> 15.539999999999997
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1

In [5]:
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1005.22
Learners:
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 1, 4] --> Action: [2, 3]
Region: [1, 0, 3, 3] --> Action: [0, 1]
Region: [0, 4, 3, 4] --> Action: [2, 3]
Region: [1, 2, 1, 4] --> Action: [0, 1]
Region: [0, 0, 3, 4] --> Action: [2, 3]
Region: [1, 2, 1, 1] --> Action: [0, 1]
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 0, 3] --> Action: [2, 3]
Region: [1, 4, 2, 4] --> Action: [0, 1]
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1009.86
Learners:
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 1, 4] --> Action: [2, 3]
Region: [1, 0, 3, 3] --> Action: [0, 1]
Region: [0, 4, 3, 4] --> Action: [2, 3]
Region: [1, 2, 1, 4] --> Action: [0, 1]
Region: [0, 0, 3, 4] --> Action: [2, 3]
Region: [1, 2, 1, 1] --> Action: [0, 1]
Region: [0

In [6]:
team = searchManager.winners[0]
for learner in team.learners:
    print(learner.program.rule.region)
env.reset()

# will end up running this 5 times...
# TODO: add up the cumulative reward 
win = False
while not win:
    win = team.q_evaluation(env)
print(win)
for learner in team.learners:
    print(f'{learner.program.rule.region} --> {learner.fitness}')

[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 0, 3]
[1, 4, 2, 4]
[0, 0, 0, 1]
[0, 0, 0, 1]
Winning action: 2
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
[0, 4, 1, 4]
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 1
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
[0, 4, 1, 4]
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 3
[0, 4, 0, 3]
Winning action: 2
[0, 4, 1, 4]
[0, 4, 3, 4]
[1, 4, 2, 4]
Winning action: 1
win!
True
0
1.2
0.9999999999999998
0
0
0.37
0
0
0.4
0
0.4
1000.1500000000001
